In [4]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

def read_keypoints_from_csv(csv_file):
    keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
            keypoints.append(frame_keypoints)
    return keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

def compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2 = read_keypoints_from_csv(csv_file2)

    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    for i in range(33):
        observation1 = np.array([keypoints1[frame1_index][i].x, keypoints1[frame1_index][i].y])
        observation2 = np.array([keypoints2[frame2_index][i].x, keypoints2[frame2_index][i].y])
        observation_distance = np.linalg.norm(observation1 - observation2)
        if observation_distance > distance_threshold:
            highlight_indices.append(i)
        '''print(f'observation_distance_{i}:', round(observation_distance, 3))
        print('#' * 100)'''
    print(highlight_indices)

    score = max(0, 100 - observation_distance)

    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    for idx in highlight_indices:
        landmark = keypoints2[frame2_index][idx]
        cv2.circle(image2, (int(landmark.x * image2.shape[1]), int(landmark.y * image2.shape[0])), 20, (0, 0, 255), -1)

    combined_image = np.concatenate((image1, image2), axis=1)

    cv2.putText(combined_image, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, f'threshold: {round(distance_threshold,3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'Points', (1700, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 0), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, image2)

# 사용 예시
video_file1 = 'C:/Users/jk/action_assess_2/data/video/Z76/313-2-1-15-Z76_D.avi'
video_file2 = 'C:/Users/jk/action_assess_2/data/video/Z106/313-2-1-15-Z106_D.avi'
csv_file1 = 'C:/Users/jk/action_assess_2/data/csv/Z76/313-2-1-15-Z76_D.csv'
csv_file2 = 'C:/Users/jk/action_assess_2/data/csv/Z106/313-2-1-15-Z106_D.csv'
output_image = 'C:/Users/jk/action_assess_2/post_data/points/output_image.png'

for threshold in np.arange(0.01, 0.10, 0.015):
    output_image = f'C:/Users/jk/action_assess_2/post_data/points/output_image_{round(threshold,3)}.png'
    compare_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=threshold)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 29, 32]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13]
[0, 1, 2, 4, 5, 6]
[]
